## Description:
这个是sharedBottom模型的demo

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import random

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat
from deepctr.feature_column import get_feature_names
from SharedBottom import SharedBottom

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True      # TensorFlow按需分配显存
config.gpu_options.per_process_gpu_memory_fraction = 0.5  # 指定显存分配比例
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

DeepCTR version 0.9.0 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.0


In [2]:
data_path = '../data_process'
data = pd.read_csv(os.path.join(data_path, 'train_data.csv'), index_col=0, parse_dates=['expo_time'])

In [3]:
# 选择出需要用到的列
use_cols = ['user_id', 'article_id', 'expo_time', 'net_status', 'exop_position', 'duration', 'device', 'city', 'age', 'gender', 'img_num', 'cat_1', 'click']
data_new = data[use_cols]

In [4]:
# 由于这个data_new的数据量还是太大， 我电脑训练不动， 所以这里再进行一波抽样
users = set(data_new['user_id'])
sampled_users = random.sample(users, 1000)
data_new = data_new[data_new['user_id'].isin(sampled_users)]

## 数据预处理

In [5]:
# 处理img_num
def transform(x):
    if x == '上海':
        return 0
    elif isinstance(x, float):
        return float(x)
    else:
        return float(eval(x))
data_new['img_num'] = data_new['img_num'].apply(lambda x: transform(x))

In [6]:
user_id_raw = data_new[['user_id']].drop_duplicates('user_id')
doc_id_raw = data_new[['article_id']].drop_duplicates('article_id')

# 简单数据预处理
sparse_features = [
    'user_id', 'article_id', 'net_status', 'exop_position', 'device', 'city', 'age', 'gender', 'cat_1'
]
dense_features = [
    'img_num'
]

# 填充缺失值
data_new[sparse_features] = data_new[sparse_features].fillna('-1')
data_new[dense_features] = data_new[dense_features].fillna(0)

# 归一化
mms = MinMaxScaler(feature_range=(0, 1))
data_new[dense_features] = mms.fit_transform(data_new[dense_features])

feature_max_idx = {}
for feat in sparse_features:
    lbe = LabelEncoder()
    data_new[feat] = lbe.fit_transform(data_new[feat])
    feature_max_idx[feat] = data_new[feat].max() + 1000

# 构建用户id词典和doc的id词典，方便从用户idx找到原始的id
# user_id_enc = data[['user_id']].drop_duplicates('user_id')
# doc_id_enc = data[['article_id']].drop_duplicates('article_id')
# user_idx_2_rawid = dict(zip(user_id_enc['user_id'], user_id_raw['user_id']))
# doc_idx_2_rawid = dict(zip(doc_id_enc['article_id'], doc_id_raw['article_id']))

In [7]:
# 划分数据集  这里按照曝光时间划分
train_data = data_new[data_new['expo_time'] < '2021-07-06']
test_data = data_new[data_new['expo_time'] >= '2021-07-06']

## 特征封装

In [8]:
sparse_feature_columns = [SparseFeat(feat, feature_max_idx[feat], embedding_dim=4) for feat in sparse_features]
Dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

In [9]:
# 划分dnn和linear特征
dnn_features_columns = sparse_feature_columns + Dense_feature_columns
lhuc_feature_columns = sparse_feature_columns

In [10]:
feature_names = get_feature_names(dnn_features_columns)

In [11]:
# AttributeError: 'numpy.dtype[int64]' object has no attribute 'base_dtype' 
# Keras需要把输入声明为Keras张量，其他的比如numpy张量作为输入不好使
train_model_input = {name: tf.keras.backend.constant(train_data[name]) for name in feature_names}
test_model_input = {name: tf.keras.backend.constant(test_data[name]) for name in feature_names}

## 模型搭建

In [22]:
model = SharedBottom(dnn_features_columns, lhuc_feature_columns, tower_dnn_hidden_units=[], task_types=['regression', 'binary'], 
             task_names=['duration', 'click'])

## 模型的训练和预测

In [13]:
label_duration = tf.keras.backend.constant(train_data['duration'].values)
label_click = tf.keras.backend.constant(train_data['click'].values)

In [14]:
# 构建数据管道
train_ds = tf.data.Dataset.from_tensor_slices((train_model_input, (label_duration, label_click))).shuffle(buffer_size=100).batch(128).prefetch(tf.data.experimental.AUTOTUNE)

In [15]:
# 模型训练这里，需要用到底层的训练脚本，这里不能用高层keras的API
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_reg_loss = tf.keras.metrics.Mean(name='train_reg_loss')
train_bin_loss = tf.keras.metrics.Mean(name='train_bin_loss')
loss_func = {"binary": tf.keras.losses.binary_crossentropy, "regression": tf.keras.losses.mean_squared_error}

In [23]:
@tf.function
def train_step(features, labels, task_types, weight=[0.02, 0.98]):
    losses = []
    
    with tf.GradientTape() as tape:
        # 遍历每个任务
        for i, task_type in enumerate(task_types):
            out = model(features, training=True)
            task_loss = loss_func[task_types[i]](out[i], labels[i])

            losses.append(weight[i] * task_loss)
            
        # 直接求和 这里可以手动指定权重
        loss = tf.add_n(losses)
        gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_reg_loss(losses[0])
    train_bin_loss(losses[1])
    return loss, losses[0], losses[1]

In [24]:
epochs = 10
best_test_loss = float('inf')
task_types = ["regression", "binary"]
for epoch in range(1, epochs+1):
    train_loss.reset_states()
    train_reg_loss.reset_states()
    train_bin_loss.reset_states()

    for feature, labels in train_ds:
        loss, loss_reg, loss_bin = train_step(feature, labels, task_types)

    template = 'Epoch {}, Loss: {} - regression_loss: {} - binary_loss: {}'
    print(template.format(epoch, train_loss.result(), 
                          np.mean(loss_reg), 
                         np.mean(loss_bin)))

Epoch 1, Loss: 140.83474731445312 - regression_loss: 161.18621826171875 - binary_loss: 5.314276218414307
Epoch 2, Loss: 138.92530822753906 - regression_loss: 161.39010620117188 - binary_loss: 5.497527599334717
Epoch 3, Loss: 138.42620849609375 - regression_loss: 126.88057708740234 - binary_loss: 5.864028453826904
Epoch 4, Loss: 138.05685424804688 - regression_loss: 111.17665100097656 - binary_loss: 4.764523029327393
Epoch 5, Loss: 137.8128204345703 - regression_loss: 127.75333404541016 - binary_loss: 4.947774410247803
Epoch 6, Loss: 137.6511993408203 - regression_loss: 109.87477111816406 - binary_loss: 5.314276218414307
Epoch 7, Loss: 137.55368041992188 - regression_loss: 237.552490234375 - binary_loss: 5.864028453826904
Epoch 8, Loss: 137.40977478027344 - regression_loss: 117.3536605834961 - binary_loss: 6.047279357910156
Epoch 9, Loss: 137.3338623046875 - regression_loss: 121.86355590820312 - binary_loss: 5.314275741577148
Epoch 10, Loss: 137.24070739746094 - regression_loss: 92.8150

In [18]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [20]:
print("test click AUC", round(roc_auc_score(test_data['click'], pred_ans[1]), 4))

test click AUC 0.47


In [21]:
print("test duration", round(mean_absolute_error(test_data['duration'], pred_ans[0]), 4))

test duration 31.7404


如果不指定权重， 就会发现， 回归任务训练的较好， 而分类任务有些差。 

In [25]:
pred_ans = model.predict(test_model_input, batch_size=256)
print("test click AUC", round(roc_auc_score(test_data['click'], pred_ans[1]), 4))
print("test duration", round(mean_absolute_error(test_data['duration'], pred_ans[0]), 4))

test click AUC 0.4914
test duration 30.4913
